In [1]:
import pandas as pd
import json
import subprocess, sys, os

## Dropping duplicate ids for BBQ safety

In [2]:
def find_duplicates(iterable):
    seen = set()
    duplicates = set()
    for item in iterable:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)
    return duplicates

In [3]:
bbq_df = pd.DataFrame(json.load(open('../../benchmarks/safety/bbq_safety.json')))
bbq_dupes = find_duplicates(list(bbq_df.id))


In [4]:
df_bbq_gpt4o_evaloptim = pd.read_csv('/mnt/bulk-titan/sanddhya/agents_edge/Agent_Edge_Benchmarking/out/evaloptimizer_gpt4o_bbq_safety/RESPONSES.csv')
df_bbq_filtered = df_bbq_gpt4o_evaloptim[~df_bbq_gpt4o_evaloptim.id.isin(bbq_dupes)]

In [5]:
print(len(df_bbq_gpt4o_evaloptim), len(df_bbq_filtered))

992 871


In [6]:
df_bbq_filtered.to_csv('FILTERED_RESPONSES_bbq_subset_gpt4o_evaloptimizer.csv')

In [8]:
def zeroshot_generate_summary(results_df, summary_path):
    total = len(results_df)
    corrects = results_df['is_correct'].sum()
    accuracy = (corrects / total) * 100 if total > 0 else 0

    acc_by_conf = results_df.groupby('confidence')['is_correct'].mean() * 100

    with open(summary_path, 'w') as f:
        f.write(f"Total questions: {total}\n")
        f.write(f"Correct: {corrects}\n")
        f.write(f"Overall Accuracy: {accuracy:.2f}%\n\n")
        f.write("Accuracy by confidence level:\n")
        for conf, acc in acc_by_conf.items():
            if pd.notna(conf):
                f.write(f"  {int(conf)}: {acc:.2f}%\n")

In [9]:
zeroshot_generate_summary(df_bbq_filtered, 'bbq_gpt4o_evaloptim_SUMMARY_FILTERED.txt')